In [17]:
import sys
# caution: path[0] is reserved for script path (or '' in REPL)
sys.path.insert(1, '../src')
sys.path.insert(1, '../prompts')
sys.path.insert(1, '../input')

from pathlib import Path
from typing import List, Dict
import os
from indexing import entity_relationship_extraction
from llama_index.core import PromptTemplate
# from models.entities import ExtractedData
# from parsers.extraction_parser import ExtractionParser
# from docling.document_converter import DocumentConverter
# from llama_index.core.node_parser import TokenTextSplitter, MarkdownNodeParser, SentenceSplitter, MarkdownElementNodeParser
# from llama_index.readers.file import FlatReader, MarkdownReader
# from llama_index.core import Document, SimpleDirectoryReader
# from llama_index.llms.ollama import Ollama


import logging
logging.basicConfig(
    level=logging.INFO,
    format='%(asctime)s - %(levelname)s - %(message)s'
)
logger = logging.getLogger(__name__)

In [2]:
from PdfProcessor import PdfProcessor
# For text output
processor = PdfProcessor(output_dir="input/txt")
processor.process_pdfs(output_format="text")

/Users/maximilianstaebler/code/PHD/RESEARCH_PAPER/eKNOW_2025/venv/lib/python3.11/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
2025-01-28 08:34:49,098 - INFO - Found 2 PDF files in ../input/pdf
2025-01-28 08:34:49,137 - INFO - Going to convert document batch...
2025-01-28 08:34:49,830 - INFO - Accelerator device: 'mps'
2025-01-28 08:34:52,274 - INFO - Accelerator device: 'mps'
2025-01-28 08:34:52,934 - INFO - Accelerator device: 'mps'
2025-01-28 08:34:53,304 - INFO - Processing document 2501.14342v1.pdf
2025-01-28 08:35:07,908 - INFO - Finished converting document 2501.14342v1.pdf in 18.81 sec.
2025-01-28 08:35:07,911 - INFO - Processed: 2501.14342v1.pdf -> 2501.14342v1.txt
2025-01-28 08:35:07,931 - INFO - Going to convert document batch...
2025-01-28 08:35:07,931 - INFO - Processing document 2408.13296v3 (1).pdf
20

In [19]:
from llama_index.llms.openai import OpenAI
llm = OpenAI(
    model="gpt-4o-mini",
    # api_key="some key",  # uses OPENAI_API_KEY env var by default
)
prompt = PromptTemplate(GRAPH_EXTRACTION_PROMPT).format(
    entity_types="ORGANIZATION,PERSON",
    input_text="The Verdantis's Central Institution is scheduled to meet on Monday and Thursday, with the institution planning to release its latest policy decision on Thursday at 1:30 p.m. PDT, followed by a press conference where Central Institution Chair Martin Smith will take questions. Investors expect the Market Strategy Committee to hold its benchmark interest rate steady in a range of 3.5%-3.75%.",
)
    

In [21]:
resp = llm.complete(prompt)
print(resp)

2025-01-28 09:35:40,557 - INFO - HTTP Request: POST https://api.openai.com/v1/chat/completions "HTTP/1.1 200 OK"


("entity"{tuple_delimiter}CENTRAL INSTITUTION{tuple_delimiter}ORGANIZATION{tuple_delimiter}The Central Institution is the Federal Reserve of Verdantis, which is responsible for setting interest rates and making monetary policy decisions.)
{record_delimiter}
("entity"{tuple_delimiter}MARTIN SMITH{tuple_delimiter}PERSON{tuple_delimiter}Martin Smith is the Chair of the Central Institution, overseeing its meetings and policy decisions.)
{record_delimiter}
("entity"{tuple_delimiter}MARKET STRATEGY COMMITTEE{tuple_delimiter}ORGANIZATION{tuple_delimiter}The Market Strategy Committee is a group within the Central Institution that focuses on interest rate policies and economic strategies.)
{record_delimiter}
("relationship"{tuple_delimiter}MARTIN SMITH{tuple_delimiter}CENTRAL INSTITUTION{tuple_delimiter}Martin Smith is the Chair of the Central Institution and will answer questions at a press conference{tuple_delimiter}9)
{record_delimiter}
("relationship"{tuple_delimiter}MARKET STRATEGY COMMITT

# GraphRAG

In [100]:
communities.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 73 entries, 0 to 72
Data columns (total 11 columns):
 #   Column             Non-Null Count  Dtype 
---  ------             --------------  ----- 
 0   id                 73 non-null     object
 1   human_readable_id  73 non-null     int64 
 2   community          73 non-null     int64 
 3   parent             73 non-null     int64 
 4   level              73 non-null     int64 
 5   title              73 non-null     object
 6   entity_ids         73 non-null     object
 7   relationship_ids   73 non-null     object
 8   text_unit_ids      73 non-null     object
 9   period             73 non-null     object
 10  size               73 non-null     int64 
dtypes: int64(5), object(6)
memory usage: 6.4+ KB


In [99]:
community_reports.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 73 entries, 0 to 72
Data columns (total 14 columns):
 #   Column             Non-Null Count  Dtype  
---  ------             --------------  -----  
 0   id                 73 non-null     object 
 1   human_readable_id  73 non-null     int64  
 2   community          73 non-null     int64  
 3   parent             73 non-null     int64  
 4   level              73 non-null     int64  
 5   title              73 non-null     object 
 6   summary            73 non-null     object 
 7   full_content       73 non-null     object 
 8   rank               73 non-null     float64
 9   rank_explanation   73 non-null     object 
 10  findings           73 non-null     object 
 11  full_content_json  73 non-null     object 
 12  period             73 non-null     object 
 13  size               73 non-null     int64  
dtypes: float64(1), int64(5), object(8)
memory usage: 8.1+ KB


In [101]:
pd.merge(
    communities, 
    community_reports, 
    on='community',           # Column to merge on
    how='outer',             # Keep all rows from both dataframes
    suffixes=('', '_reports'), # Add suffixes to distinguish duplicate column names
).info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 73 entries, 0 to 72
Data columns (total 24 columns):
 #   Column                     Non-Null Count  Dtype  
---  ------                     --------------  -----  
 0   id                         73 non-null     object 
 1   human_readable_id          73 non-null     int64  
 2   community                  73 non-null     int64  
 3   parent                     73 non-null     int64  
 4   level                      73 non-null     int64  
 5   title                      73 non-null     object 
 6   entity_ids                 73 non-null     object 
 7   relationship_ids           73 non-null     object 
 8   text_unit_ids              73 non-null     object 
 9   period                     73 non-null     object 
 10  size                       73 non-null     int64  
 11  id_reports                 73 non-null     object 
 12  human_readable_id_reports  73 non-null     int64  
 13  parent_reports             73 non-null     int64  
 

In [ ]:
import pandas as pd

entities = pd.read_parquet('../ragtest/output/create_final_entities.parquet')

entities.head(5)

,id,human_readable_id,title,type,description,text_unit_ids
0,238b7bf8-e7c7-4936-b813-31acd13cfe9b,0,VENKATESH BALAVADHANI PARTHASARATHY,PERSON,Venkatesh Balavadhani Parthasarathy is one of ...,[9ed73c3fa03f83895a135beea0b9167417e1259afd8d6...
1,59febcce-d32d-44e4-9527-9f3a0f5d38aa,1,AHTSHAM ZAFAR,PERSON,Ahtsham Zafar is an author recognized for his ...,[9ed73c3fa03f83895a135beea0b9167417e1259afd8d6...
2,6f1a5ad5-05a6-4219-93bc-df947fcba962,2,AAFAQ KHAN,PERSON,Aafaq Khan is one of the authors of the techni...,[9ed73c3fa03f83895a135beea0b9167417e1259afd8d6...
3,bfc913d9-6ef6-47c9-bd43-c511849bd03d,3,ARSALAN SHAHID,PERSON,Arsalan Shahid is one of the authors of the te...,[9ed73c3fa03f83895a135beea0b9167417e1259afd8d6...
4,331685e5-74fa-4788-867e-0aa6b0a5c8d6,4,CEADAR,ORGANIZATION,"CeADAR is Ireland's Centre for AI, affiliated ...",[9ed73c3fa03f83895a135beea0b9167417e1259afd8d6...


In [5]:
relationships = pd.read_parquet('../ragtest/output/create_final_relationships.parquet')

relationships.head()

,id,human_readable_id,source,target,description,weight,combined_degree,text_unit_ids
0,24e7b520-b15a-4b91-9f58-f58eda6a4a9c,0,VENKATESH BALAVADHANI PARTHASARATHY,CEADAR,Venkatesh Balavadhani Parthasarathy is affilia...,7.0,8,[9ed73c3fa03f83895a135beea0b9167417e1259afd8d6...
1,4c1cc3e2-63c3-481d-9ffc-96464bbd6af8,1,VENKATESH BALAVADHANI PARTHASARATHY,FINE-TUNING LLMS,Venkatesh Balavadhani Parthasarathy contribute...,8.0,7,[9ed73c3fa03f83895a135beea0b9167417e1259afd8d6...
2,02aada02-0058-407c-abe1-a149135c10b3,2,AHTSHAM ZAFAR,CEADAR,Ahtsham Zafar is affiliated with CeADAR as an ...,7.0,9,[9ed73c3fa03f83895a135beea0b9167417e1259afd8d6...
3,e7903602-1e7c-4ed4-a676-1cfd91e8c9ea,3,AHTSHAM ZAFAR,FINE-TUNING LLMS,Ahtsham Zafar contributed to the technical rep...,8.0,8,[9ed73c3fa03f83895a135beea0b9167417e1259afd8d6...
4,2920545e-3a8b-450a-b335-5b21da426947,4,AAFAQ KHAN,CEADAR,Aafaq Khan is affiliated with CeADAR as an aut...,7.0,8,[9ed73c3fa03f83895a135beea0b9167417e1259afd8d6...


In [78]:
nodes = pd.read_parquet('../ragtest/output/create_final_nodes.parquet')

nodes.head(5)

,id,human_readable_id,title,community,level,degree,x,y
0,238b7bf8-e7c7-4936-b813-31acd13cfe9b,0,VENKATESH BALAVADHANI PARTHASARATHY,-1,0,2,0.0,0.0
1,59febcce-d32d-44e4-9527-9f3a0f5d38aa,1,AHTSHAM ZAFAR,-1,0,3,0.0,0.0
2,6f1a5ad5-05a6-4219-93bc-df947fcba962,2,AAFAQ KHAN,-1,0,2,0.0,0.0
3,bfc913d9-6ef6-47c9-bd43-c511849bd03d,3,ARSALAN SHAHID,-1,0,2,0.0,0.0
4,331685e5-74fa-4788-867e-0aa6b0a5c8d6,4,CEADAR,-1,0,6,0.0,0.0


In [93]:
community_reports = pd.read_parquet('../ragtest/output/create_final_community_reports.parquet')

community_reports.sort_values(by='human_readable_id', ascending=True).head(5)

,id,human_readable_id,community,parent,level,title,summary,full_content,rank,rank_explanation,findings,full_content_json,period,size
56,a9effcbed22b47b081b6c6014ce7a9cc,0,0,-1,0,CoRAG and KILT Community,The community centers around advanced multi-ho...,# CoRAG and KILT Community\n\nThe community ce...,8.5,The impact severity rating is high due to the ...,[{'explanation': 'CoRAG is a comprehensive fra...,"{\n ""title"": ""CoRAG and KILT Community"",\n ...",2025-01-28,43
57,be1e1c0057f84bedbe35b202fc6068e2,1,1,-1,0,AI Models and Performance Metrics,The community focuses on AI models and their a...,# AI Models and Performance Metrics\n\nThe com...,7.5,The impact severity rating is high due to the ...,[{'explanation': 'AI models are the core of th...,"{\n ""title"": ""AI Models and Performance Met...",2025-01-28,24
58,770f6bbe548240e5bf65dadcb375f9aa,2,2,-1,0,Monitoring Programme for LLM Applications,The community focuses on the Monitoring Progra...,# Monitoring Programme for LLM Applications\n\...,7.5,The impact severity rating is high due to the ...,[{'explanation': 'The Monitoring Programme is ...,"{\n ""title"": ""Monitoring Programme for LLM ...",2025-01-28,5
59,2e8f07a49e3e442fbe96b04f94e497b2,3,3,-1,0,Autotrain and Machine Learning Ecosystem,"The community centers around Autotrain, a plat...",# Autotrain and Machine Learning Ecosystem\n\n...,7.5,The impact severity rating is high due to the ...,[{'explanation': 'Autotrain is a pivotal entit...,"{\n ""title"": ""Autotrain and Machine Learnin...",2025-01-28,27
60,180a8f416fd1402cad4030f8deb8f340,4,4,-1,0,Data Annotation and Processing Community,The community focuses on various aspects of da...,# Data Annotation and Processing Community\n\n...,7.5,The impact severity rating is high due to the ...,[{'explanation': 'Data preprocessing is a cruc...,"{\n ""title"": ""Data Annotation and Processin...",2025-01-28,15


In [66]:
documents = pd.read_parquet('../ragtest/output/create_final_documents.parquet')

documents.head(10)

,id,human_readable_id,title,text,text_unit_ids
0,a8b765d9d64c173b527b19399d8071b2476bc8aa345827...,1,2408.13296v3 (1).txt,<missing-text>\n\nThe Ultimate Guide to Fine-T...,[9ed73c3fa03f83895a135beea0b9167417e1259afd8d6...
1,f7ed019e71d299d7b11947cc8c588531072e3b2514bc47...,2,2501.14342v1.txt,Chain-of-Retrieval Augmented Generation\n\nLia...,[9241e4bd9bdbdadd0dbb7c30415c3cf8f82547f41b833...


In [71]:
[len(x) for x in documents.text_unit_ids.values]

[41, 12]

In [91]:
communities = pd.read_parquet('../ragtest/output/create_final_communities.parquet')

communities.head(5)

,id,human_readable_id,community,parent,level,title,entity_ids,relationship_ids,text_unit_ids,period,size
0,cd563348-1f2d-4918-a073-27204e982cfa,0,0,-1,0,Community 0,"[001d4015-f116-4c3c-b21b-376032a30072, 0b21231...","[0ccf1db1-d739-42b2-8775-2034df86ab2d, 0ced99d...",[abfaf87b2fdd52560c6a4a3c59178b5415b4aa9633993...,2025-01-28,43
1,30f48ce0-baf3-432c-a54e-0fcff152e92a,1,1,-1,0,Community 1,"[89525cf5-bd1d-4806-8517-8595539606d6, a356eb3...","[00277c37-57fc-417d-8e23-89a50fe3df1e, 0b3aca4...",[37d97c622365f841418608936d8c4aed1956f90742820...,2025-01-28,24
2,4c050187-4fe2-4bde-b241-672023ff5493,2,2,-1,0,Community 2,"[4ab0bd77-514a-48f6-963d-2cdbe3589251, 20446e9...","[09f8ace1-1917-4e5b-886e-4604b37d228c, 110d6d9...",[38a6728aac6bcf737e45a075a16b7faf71404c0ee615c...,2025-01-28,5
3,c5117d9e-8926-4ba1-ad1c-f6be37618e1a,3,3,-1,0,Community 3,"[425a9730-89d0-49d5-9ddc-1921ef31502c, 3c121ea...","[0ef0e575-977a-4d26-ba42-58c510460b38, 1c69d5f...",[07f3c83a7132ae24df43eb0fc6755313b2f284bdd8cfa...,2025-01-28,27
4,37bb8d03-e3ae-438b-abef-b6786337fef4,4,4,-1,0,Community 4,"[597f7281-d668-4a6f-bc84-d3368ae2be5b, 9ce3a72...","[0af4b42b-a06f-4414-ab44-c03cfdcc7e59, 2f6148c...",[06626a8bf437d9ff039f9da18900405390479566802f5...,2025-01-28,15


In [87]:
len(communities.human_readable_id.unique())
a9effcbed22b47b081b6c6014ce7a9cc

73

In [79]:
text_units = pd.read_parquet('../ragtest/output/create_final_text_units.parquet')

text_units.head(5)

,id,human_readable_id,text,n_tokens,document_ids,entity_ids,relationship_ids
0,9ed73c3fa03f83895a135beea0b9167417e1259afd8d6a...,1,<missing-text>\n\nThe Ultimate Guide to Fine-T...,1200,[a8b765d9d64c173b527b19399d8071b2476bc8aa34582...,"[238b7bf8-e7c7-4936-b813-31acd13cfe9b, 59febcc...","[e9547e87-ccdc-48a0-856c-7a95b7f70474, e86e48a..."
1,adb58fb2ac4dc2e636420a23765cd5f718dd4f88e00555...,2,ω i | ω 1 ω 2 · · · ω i -1 ) = C ( ω 1 ω 2 · ...,1200,[a8b765d9d64c173b527b19399d8071b2476bc8aa34582...,"[9e34ec08-09ca-4f9d-8468-7db4d67e4fe5, f6b0e00...","[40e58aa5-9897-42e8-a2ac-9563229c25db, 4a544c2..."
2,759dea32dd8bddec032bdfe47d91384a521198df103896...,3,domain.\n\n 7. Faster Convergence: Fine-tunin...,1200,[a8b765d9d64c173b527b19399d8071b2476bc8aa34582...,"[415a36ba-3521-46ba-9ba1-5711e1ae54a8, d194007...","[eb380531-a17d-4e53-bcf5-13fe7238d090, 33c948b..."
3,fd707bba863fc24f3213c4f6a01fc8e26336f84b4e4b2a...,4,", including model initialisation, hyperparamet...",1200,[a8b765d9d64c173b527b19399d8071b2476bc8aa34582...,"[122ebd66-2423-4a18-8883-97bce1b1848f, 206c08c...","[63e56d3c-1441-4641-8129-1ad2a4f8c43f, c0f3a2d..."
4,d71a1297e87b844c19e7760a007680aa86930cbaed36e0...,5,used for data collection.\n\n3.1.2 Data Prepr...,1200,[a8b765d9d64c173b527b19399d8071b2476bc8aa34582...,"[83b181e3-94cc-4a7b-a0d5-78c754dc85f0, c5005d1...","[dcd408ad-cb65-4fcc-b273-f7cecc4ef16c, 60c97b8..."


In [83]:
text_units.relationship_ids.values[0]

array(['e9547e87-ccdc-48a0-856c-7a95b7f70474',
       'e86e48ad-9c42-4e5f-8402-5bd01350785d',
       '2f58e1bf-19c5-4076-ac21-a9d1a6220f5b',
       '4a775bdc-cf9f-47b5-bc57-f24dfb17c58d',
       'ece68564-a029-488e-906c-c7458ac4a7c6',
       'b829359b-5fe2-4492-a5ae-2618303a18d3',
       '74995cbd-28fb-4ca9-a267-017b6d08127b',
       '116311e8-ec45-41b8-8f0c-b6ae6f1970dd',
       'cd309ac7-a2c4-4393-a4f7-044eb7260b8b',
       '56ffc6b4-ec88-4ec5-9b0a-dc75efdfd37a'], dtype=object)

In [8]:
print(community_reports["full_content"][0])

# Hyperparameter Tuning and Adjustments

The community focuses on hyperparameter tuning, a vital process in AI and machine learning, and its relationship with adjustments made to optimize model performance. The entities are interconnected, with adjustments being a critical component of the hyperparameter tuning process.

## Importance of Hyperparameter Tuning

Hyperparameter tuning is a critical process in artificial intelligence and machine learning that involves adjusting key parameters within models to enhance their performance. This process is essential for achieving optimal performance during the training phase of machine learning models, as it focuses on optimizing the parameters that govern the training process. By fine-tuning these hyperparameters, practitioners aim to improve the overall effectiveness and efficiency of AI models. The significance of this process cannot be overstated, as it directly impacts the quality of AI outputs and the reliability of machine learning appli

In [9]:
print(community_reports["summary"][0])

The community focuses on hyperparameter tuning, a vital process in AI and machine learning, and its relationship with adjustments made to optimize model performance. The entities are interconnected, with adjustments being a critical component of the hyperparameter tuning process.


In [10]:
import subprocess
import shlex
from typing import Optional

def query_graphrag(
    query: str,
    method: str = "global",
    root_path: str = "../ragtest",
    timeout: Optional[int] = None,
    community_level: int = 2,
    dynamic_community_selection: bool = False
) -> str:
    """
    Execute a GraphRAG query using the CLI tool.
    
    Args:
        query (str): The query string to process
        method (str): Query method (e.g., "global", "local", or "drift")
        root_path (str): Path to the root directory
        timeout (int, optional): Timeout in seconds for the command
        community_level (int): The community level in the Leiden community hierarchy (default: 2)
        dynamic_community_selection (bool): Whether to use global search with dynamic community selection (default: False)
    
    Returns:
        str: The output from GraphRAG
        
    Raises:
        subprocess.CalledProcessError: If the command fails
        subprocess.TimeoutExpired: If the command times out
        ValueError: If community_level is negative
    """
    # Validate community level
    if community_level < 0:
        raise ValueError("Community level must be non-negative")
    
    # Construct the base command
    command = [
        'graphrag', 'query',
        '--root', root_path,
        '--method', method,
        '--query', query,
        '--community-level', str(community_level)
    ]
    
    # Add dynamic community selection flag if enabled
    if dynamic_community_selection:
        command.append('--dynamic-community-selection')
    
    try:
        # Execute the command and capture output
        result = subprocess.run(
            command,
            capture_output=True,
            text=True,
            timeout=timeout
        )
        
        # Check if the command was successful
        result.check_returncode()
        
        return result.stdout.strip()
        
    except subprocess.CalledProcessError as e:
        error_message = f"Command failed with exit code {e.returncode}\nError: {e.stderr}"
        raise subprocess.CalledProcessError(
            e.returncode,
            e.cmd,
            output=e.output,
            stderr=error_message
        )

In [11]:
result = query_graphrag(
    query="How does a company choose between RAG, fine-tuning, and different PEFT approaches?",
    method="local"
)
print("Query result:")
print(result)

Query result:
INFO: Vector Store Args: {
    "type": "lancedb",
    "db_uri": "/Users/maximilianstaebler/code/PHD/RESEARCH_PAPER/eKNOW_2025/ragtest/output/lancedb",
    "collection_name": "default",
    "overwrite": true
}
creating llm client with {'api_key': 'REDACTED,len=164', 'type': "openai_chat", 'encoding_model': 'cl100k_base', 'model': 'gpt-4o-mini', 'max_tokens': 4000, 'temperature': 0.0, 'top_p': 1.0, 'n': 1, 'frequency_penalty': 0.0, 'presence_penalty': 0.0, 'request_timeout': 180.0, 'api_base': None, 'api_version': None, 'organization': None, 'proxy': None, 'audience': None, 'deployment_name': None, 'model_supports_json': True, 'tokens_per_minute': 50000, 'requests_per_minute': 1000, 'max_retries': 10, 'max_retry_wait': 10.0, 'sleep_on_rate_limit_recommendation': True, 'concurrent_requests': 25, 'responses': None}
creating embedding llm client with {'api_key': 'REDACTED,len=164', 'type': "openai_embedding", 'encoding_model': 'cl100k_base', 'model': 'text-embedding-3-small', 

In [65]:
result = query_graphrag(
    query="How does a company choose between RAG, fine-tuning, and different PEFT approaches?",
    method="local"
)
print("Query result:")
print(result)

Query result:
INFO: Vector Store Args: {
    "type": "lancedb",
    "db_uri": "/Users/maximilianstaebler/code/PHD/RESEARCH_PAPER/eKNOW_2025/ragtest/output/lancedb",
    "collection_name": "default",
    "overwrite": true
}
creating llm client with {'api_key': 'REDACTED,len=164', 'type': "openai_chat", 'encoding_model': 'cl100k_base', 'model': 'gpt-4o-mini', 'max_tokens': 4000, 'temperature': 0.0, 'top_p': 1.0, 'n': 1, 'frequency_penalty': 0.0, 'presence_penalty': 0.0, 'request_timeout': 180.0, 'api_base': None, 'api_version': None, 'organization': None, 'proxy': None, 'audience': None, 'deployment_name': None, 'model_supports_json': True, 'tokens_per_minute': 50000, 'requests_per_minute': 1000, 'max_retries': 10, 'max_retry_wait': 10.0, 'sleep_on_rate_limit_recommendation': True, 'concurrent_requests': 25, 'responses': None}
creating embedding llm client with {'api_key': 'REDACTED,len=164', 'type': "openai_embedding", 'encoding_model': 'cl100k_base', 'model': 'text-embedding-3-small', 